# 主題：政治新聞資料分析
- 組員：王冠人、萬俊彥、廖品琪
- 時間：2019/01/29

## 研究目標
考量臺灣社會政治冷感的現狀，提供一個較為客觀的資料統整，藉由日常生活中的事件觀察政治人物的網路聲量變化，作為一種參與政治的方式。

## 研究方法
1. 爬蟲搜尋網路新聞
2. 進行資料清理及資料分析
4. 視覺化呈現研究結果

## 研究內容及結果

# Part I 爬蟲

- [新頭殼新聞爬蟲](https://github.com/MiccWan/Political-News-Analysis/blob/master/crawler/new_talk_crawler.ipynb)
- [自由時報新聞爬蟲](https://github.com/MiccWan/Political-News-Analysis/blob/master/crawler/liberty_times_crawler.ipynb)

#  Part II 資料清洗及分析

In [1]:
import os
import pickle
import jieba
import operator
import statistics
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.font_manager as font_manager
from datetime import datetime
from collections import Counter

from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
from PIL import Image

from modules import *

# 載入外部字型
font_path = '../tools/msjh.ttc'
font = font_manager.FontProperties(fname='../tools/msjh.ttc',
                                   weight='bold',
                                   style='normal', size=16)

### 載入新聞資料

In [2]:
# 讀取先前爬好的新頭殼新聞文字稿
with open('../crawler/data/new_talk.pkl', 'rb') as f:
    data = pickle.load(f)

# 將新聞的順序倒序
data = data[::-1]

# 將新聞的內容彙整成列表
contents = [news['content'] for news in data]

### 使用 coshow 找出新詞 -> 自定義辭典

In [3]:
# 用get_coshow把新聞內容前1000篇進行清理與分詞，並統計詞頻
get_coshow(contents[:1000])[:10]

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\Stella\AppData\Local\Temp\jieba.cache
Loading model cost 0.883 seconds.
Prefix dict has been built succesfully.


[('68', 1002),
 ('蔡英文', 775),
 ('民進黨', 609),
 ('對於', 577),
 ('柯文哲', 547),
 ('台北市長', 388),
 ('沒有', 312),
 ('）日', 279),
 ('長參選人', 257),
 ('記者會', 254)]

### 設定繁體中文斷詞、載入自訂辭典、stopwords

In [4]:
# 載入外部辭典與自訂辭典
jieba.set_dictionary('../jieba_data/dict.txt.big')
jieba.load_userdict('../jieba_data/userdict.txt')

# 建立stopwords列表
stopwords = []
with open('../jieba_data/stopwords.txt', 'r', encoding='UTF-8') as file:
    for each in file.readlines():
        stopwords.append(each.strip())
    stopwords.append(' ')

Building prefix dict from C:\Users\Stella\Desktop\假D槽\碩一課程\下學期\金融科技與文字探勘\Political-News-Analysis-master\jieba_data\dict.txt.big ...
Loading model from cache C:\Users\Stella\AppData\Local\Temp\jieba.ue9f15f1f7a27a2a408ab3f5b910b0d6a.cache
Loading model cost 1.518 seconds.
Prefix dict has been built succesfully.


In [5]:
# 將每篇新聞清理、斷詞，統計詞頻
for i in range(len(data)):
    current_content = data[i]['content']
    current_cutted = jieba.lcut(remove_punctuation(current_content))
    data[i]['cutted_dict'] = lcut_to_dict(current_cutted)
    

In [6]:
# 找出常一起出現的詞彙
get_coshow(contents[:1000])[:10]

[('68', 1002),
 ('）日', 333),
 ('今（', 330),
 ('都是', 253),
 ('台灣的', 236),
 ('也是', 207),
 ('8月', 194),
 ('（）', 182),
 ('為了', 181),
 ('的人', 179)]

In [7]:
#把前1000篇新聞清理、斷詞、統計詞頻，找出詞頻前20高的詞為high_freq_pair
cutted_dict = get_cutted_dict(contents[:1000])
high_freq_pair = first_n_words(cutted_dict, 30)
high_freq_pair

[('台灣', 1789),
 ('中國', 1014),
 ('國民黨', 903),
 ('柯文哲', 836),
 ('民進黨', 640),
 ('指出', 597),
 ('媒體', 553),
 ('希望', 543),
 ('一個', 446),
 ('政府', 445),
 ('蔡英文', 435),
 ('政策', 425),
 ('支持', 411),
 ('提供', 401),
 ('國家', 401),
 ('未來', 401),
 ('姚文智', 376),
 ('選舉', 371),
 ('提出', 369),
 ('市長', 369),
 ('市府', 347),
 ('活動', 347),
 ('國際', 346),
 ('政治', 343),
 ('總統', 336),
 ('針對', 336),
 ('台北', 321),
 ('蘇貞昌', 320),
 ('臉書', 318),
 ('議員', 315)]

In [8]:
# 把全部新聞清理、斷詞、統計詞頻
cutted_dict = get_cutted_dict(contents)

In [9]:
# 用first_n_words找出可能人名(詞長為3個字)
possible_name = first_n_words(cutted_dict, 1000, 3, 3)
possible_name[:10]

[('民進黨', 8663),
 ('柯文哲', 8359),
 ('國民黨', 8188),
 ('韓國瑜', 8007),
 ('陳其邁', 4266),
 ('蔡英文', 4011),
 ('蘇貞昌', 3261),
 ('候選人', 2955),
 ('姚文智', 2456),
 ('賴清德', 2202)]

In [10]:
# 用first_n_words找出可能事件(詞長大於4個字)
possible_events = first_n_words(cutted_dict, 200, 4)
possible_events[:10]

[('台北市長柯文哲', 1557),
 ('總統蔡英文', 1410),
 ('九二共識', 1136),
 ('時代力量', 1097),
 ('立委補選', 1090),
 ('中華民國', 950),
 ('2020', 918),
 ('行政院長賴清德', 865),
 ('轉型正義', 814),
 ('競選總部', 766)]

### 載入人名、事件

In [11]:
# 從外部檔案讀入政治人物名字
names = []
with open('../data/names.txt', 'r', encoding='utf-8-sig') as f:
    names = f.read().split('\n')
# 從外部檔案讀入政治事件    
events = []
with open('../data/events.txt', 'r', encoding='utf-8-sig') as f:
    events = f.read().split('\n')

In [12]:
# 看一下內容
print(' '.join(names[:5]))
print(' '.join(events[:5]))

柯文哲 韓國瑜 陳其邁 蔡英文 蘇貞昌
九二共識 兩岸關係 轉型正義 立委補選 九合一選舉
